In [ ]:
from dotenv import load_dotenv
import os
import re
import os
import pandas as pd
import numpy as np
from groq import Groq
from stockfish import Stockfish

In [ ]:
load_dotenv()  # Load variables from .env
STOCKFISH_PATH=os.environ.get("STOCKFISH_PATH")
LLAMA_MODEL_PATH=os.environ.get("LLAMA_MODEL_PATH")

In [ ]:
def clean(response):
    response=re.sub(r'<think>.*?</think>','',response,flags=re.DOTALL)
    return response

In [ ]:
stockfish = Stockfish(STOCKFISH_PATH)
def get_best_move(fen):
    stockfish.set_fen_position(fen)
    return stockfish.get_best_move()

In [ ]:
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)
def chat_complete(fen,best_move,type_of_move):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a chess coach , explain it well as if you thought of the move and try to focus more on the move and why it is the best move, be extremely nice to the student",
            },
            {
                "role": "user",
                "content": f"Justify why {type_of_move} {best_move} is the best move for the FEN {fen}",
            }
        ],
        model="deepseek-r1-distill-llama-70b",
    )

    return clean(chat_completion.choices[0].message.content)

In [ ]:
def bm_w_exp():
    fen = input("You : ")
    best_move = get_best_move(fen)
    g = str(stockfish.will_move_be_a_capture(best_move)).split(".")[1]
    if g=="NO_CAPTURE":
        g=False
    else:
        g=True
    p1_split = str(stockfish.get_what_is_on_square(best_move[:2])).split(".")
    p2_split = str(stockfish.get_what_is_on_square(best_move[2:])).split(".")
    p1 = p1_split[1] if len(p1_split) > 1 else p1_split[0]
    p2 = p2_split[1] if len(p2_split) > 1 else p2_split[0]
    if g:
        type_of_move = f"Capturing the {p2} with {p1}"
    elif "e8g8" in best_move and "KING" in p1:
        type_of_move = f"Castling with the {p1}"
    elif "e8b8" in best_move and "KING" in p1:
        type_of_move = f"Castling with the {p1}"
    elif "e1g1" in best_move and "KING" in p1:
        type_of_move = f"Castling with the {p1}"
    elif "e1b1" in best_move and "KING" in p1:
        type_of_move = f"Castling with the {p1}"
    else:
        type_of_move = "Move"
    print("Bot : ", chat_complete(fen, best_move, type_of_move))